# 2022/06/17: 学問への扉（音と画像のデジタル処理）
--- 

画像の圧縮についての実験をします．  
【発展】と書いてある部分は，余裕があればやってみてください．

## 画像の読み込み
---

[OpenCV](https://opencv.org)を使って画像を読み込みます．
- [OpenCVの公式ドキュメント（英語）](https://docs.opencv.org/4.3.0/)

> **実験**  
> 1. `# 初期設定` から始まる下のブロックのプログラムをそのまま実行しましょう．ブロックにカーソルを合わせると左上に三角の再生ボタンのようなものが出てくるので，そこをクリックするとプログラムを実行できます．左にチェックマークがついたら実行が完了しています．
> 1. その下の `# 以下を実行すると〜` から始まるブロックのプログラムを実行しましょう．「ファイル選択（Choose Files）」をクリックして好きな画像をアップロードしてください（他人には公開されません）．
> 自分が持っている画像でも良いですし，素材サイト（https://www.pakutaso.com など）からダウンロードした画像でも構いません．


In [ ]:
# 初期設定
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import cv2 # opencvをインポート


# 関数をいくつか定義
def load_image():
    """
    画像を読み込む関数

    """
    uploaded_file = files.upload()
    file_name = next(iter(uploaded_file)) # ファイル名取得
    print('file name =', file_name)
    img = cv2.imread(file_name) # 画像読み込み

    return img


def show_image(img, title='', fontsize=20): 
    """
    画像を表示する関数

    入力
    ----------
    img: 配列
        表示したい画像
    title: 文字列
        画像のタイトル（なくてもよい）
    fontsize: 整数
        タイトルのフォントサイズ（デフォルト：20）
    
    """
    
    fig, ax = plt.subplots()
    img_converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img_converted)
    ax.axis('off')
    ax.set_title(title, fontsize=fontsize)
    plt.show()


def compress_image(img, quality):
    """
    画像を圧縮する関数

    入力
    ----------
    img: 配列
        圧縮したい画像
    quality: 0-100
        結果の画像の質（数値が大きいほど画質が良い，小さいほど圧縮できる）
    
    """
    param = [cv2.IMWRITE_JPEG_QUALITY, quality]
    _, buf = cv2.imencode('.jpg', img, param)
    img_new = cv2.imdecode(buf, 1)

    return img_new


In [ ]:
# 以下を実行するとファイルをアップロードするフォームが出てくるので
# 「ファイル選択（Choose Files）」から画像ファイルをアップロードする
# （画像サイズが大きいと時間がかかるので，5MB程度までが無難）
img = load_image() # imgという変数に画像の情報が格納される

## JPEG圧縮
---

画像の圧縮を行い，圧縮度合いと画質の関係を見てみましょう．


> **実験**  
> 1. 下のブロックのプログラムをそのまま実行してみましょう．
> 圧縮前の画像と圧縮後の画像が表示されます．
> 2. `quality`という値を変えると，どれくらい圧縮するかが変わります．
> `quality`は0から100までの値をとることができて，数値が大きいほど結果の画質が良く（圧縮度合いが小さく）なり，数値が小さいほど画質が悪く（圧縮度合いが大きく）なります．  
> `quality=20`となっているところの数値をいろいろ変えてプログラムを実行してみて，結果がどのようになるか観察しましょう．
> 3. 一つ上のブロックを再度実行して先ほどとは別の画像を読み込み，その画像を圧縮したときの結果がどうなるかを見てみましょう．

In [ ]:
show_image(img, 'before')  # 元の画像を表示

# 圧縮
quality = 20  # 圧縮後の画像の画質の設定
img_compressed = compress_image(img, quality)  # 圧縮後の画像をimg_compressedという変数に保存
show_image(img_compressed, f'after (quality={quality})')  # 圧縮後の画像を表示

## 画像の保存
---

Googleドライブと連携して，画像ファイルを保存しましょう．

> **実験**  
> 1. 下の2つのブロックのプログラムを順番に実行し，圧縮後の画像がGoogleドライブに保存されることを確認しましょう．
> 2. 一つ前のブロックのプログラムを`quality`の値を変えてから実行し，その後下の保存のプログラムを実行すると，その`quality`の値に対する圧縮後の画像がGoogleドライブに保存されます．  
> 様々な`quality`の値に対応する圧縮後の画像を保存しましょう．
> 3. Googleドライブから圧縮後の画像をダウンロードし，それぞれのファイルサイズを確認してください．
> qualityの値とファイルサイズの間にどのような関係があるかを見てみましょう．

In [ ]:
from google.colab import drive

# Googleドライブをマウント（自分のGoogleアカウントのGoogleドライブにプログラムからアクセスできるようにする）
# 何か許可を求められたら許可してください
drive.mount('/content/drive')

In [ ]:
# 変数img_compressedに入っている画像をcompressed(quality=【qualityの値】).jpgとしてGoogleドライブの「マイドライブ」に保存
cv2.imwrite(f'/content/drive/My Drive/compressed(quality={quality}).jpg', img_compressed)

## 【発展】圧縮画像の画質の評価

---

先ほどまでは目で見て画像がきれいかどうか判断していましたが，画像の画質を定量的に評価することもできます．  
時間が余ったらやってみてください．

> **実験**  
> 1. PSNR（Peak Signal to Noise Ratio）という指標を使うと，圧縮前の画像と圧縮後の画像がどれくらい近いかを評価することができます．PSNRの値が大きいほど2つの画像が近いことを示しています．  
> 下のブロックのプログラムをそのまま実行して，PSNRの値を表示してみましょう．
> 2. `quality`の値を変えたときの圧縮後の画像を上の方のブロックのプログラムを使って計算し，その画像のPSNRを求めましょう．

In [ ]:
# img_compressedに入っている画像のPSNRを表示
print('PSNR:', cv2.PSNR(img, img_compressed))

## 【さらに発展】圧縮率と画質の関係をグラフ化

---

qualityや圧縮率と画質の関係をグラフにすると，実験結果を視覚的に理解しやすくなります．


> **実験**  
> 1. 下のプログラムをそのまま実行しましょう．`x`という配列の中の数値を横軸，`y`という配列の中の数値を縦軸としてプロットしたグラフが表示されます．
> 1. `x`や`y`の配列の中身を変更して，横軸を`quality`の値，縦軸を圧縮後の画像のファイルサイズにしたグラフを作成しましょう．ファイルサイズは，先ほどの実験と同様にGoogleドライブから圧縮後の画像ファイルをダウンロードして確認してください．
> 1. `x`や`y`の配列の中身を変更して，横軸を`quality`の値，縦軸を圧縮後の画像のPSNRの値にしたグラフを作成しましょう．PSNRの値は一つ上の実験と同じようにして確認しましょう．
> 1. 【pythonプログラミングができる人向け】プロットするPSNRを手で入力するのではなく，`quality`の値の配列から対応するPSNRの値の配列を作成するプログラムを作成し，その結果を用いて`quality`とPSNRの関係のグラフを表示しましょう．

In [ ]:
# プロットする値の配列
x = np.array([0, 1, 3, 5, 10])
y = np.array([1, 4, 4, 8, 0])
# 軸のラベル
x_label = 'x'
y_label = 'y'

# プロット
fig, ax = plt.subplots()
ax.plot(x, y)
ax.set_xlabel(x_label, fontsize=16)
ax.set_ylabel(y_label, fontsize=16)
ax.tick_params(labelsize=16)
ax.grid(linestyle=':')
plt.show()